# SCPI server example

This notebook shows how to create a simple SCPI server using `workbench.utils.server` so you can expose your own instruments and devices over TCP.


## Define your instrument

Subclass `ScpiInstrument` and register SCPI commands with `@scpi_command`. Basic versions of standard commands like `*IDN?`, `*RST`, and `SYST:ERR?` are already implemented by `ScpiInstrument`.


In [ ]:
from workbench.utils.server import ScpiInstrument, scpi_command, SCPIError


class DemoSupply(ScpiInstrument):
    def __init__(self):
        super().__init__(identity="Acme,DemoSupply,1234,1.0")
        self.voltage = 5.0
        self.output_enabled = False

    @scpi_command(r"^SOUR(?:ce)?:VOLT(?:age)?\s+(\S+)$")
    def set_voltage(self, value):
        try:
            self.voltage = float(value)
        except ValueError:
            raise SCPIError(-104, "Data type error")

    @scpi_command(r"^SOUR(?:ce)?:VOLT(?:age)?\?$")
    def get_voltage(self):
        return f"{self.voltage:.3f}"

    @scpi_command(r"^OUTP(?:ut)?\s+(ON|OFF)$")
    def set_output(self, state):
        self.output_enabled = state.upper() == "ON"

    @scpi_command(r"^OUTP(?:ut)?\?$")
    def get_output(self):
        return "ON" if self.output_enabled else "OFF"

    @scpi_command(r"^MEAS(?:ure)?:VOLT(?:age)?\?$")
    def measure_voltage(self):
        if not self.output_enabled:
            raise SCPIError(-221, "Settings conflict")
        return f"{self.voltage:.3f}"


## Start the server (command line)

The cleanest way to run the server is from a script in a separate terminal. Save your device class in a module (for example `demo_supply.py`) and run a small launcher:

```python
from workbench.utils.server import ScpiServer
from demo_supply import DemoSupply

server = ScpiServer(DemoSupply())
server.start(host="127.0.0.1", port=5025)
```

Then start it with:

```bash
python run_scpi_server.py
```

Keep that running while you connect from clients.


In [2]:
from threading import Thread
from workbench.utils.server import ScpiServer

server = ScpiServer(DemoSupply())
thread = Thread(target=server.start, args=("127.0.0.1", 5025))
thread.start()

## Talk to the server with PyVISA

Use `VisaInstrument` to connect with PyVISA. Make sure the server is running first.


In [ ]:
from workbench.instruments import VisaInstrument


class DemoSupplyClient(VisaInstrument):
    pass


with DemoSupplyClient.connect("TCPIP::127.0.0.1::5025::SOCKET") as instrument:
    print(instrument.query("*IDN?"))
    instrument.write("SOUR:VOLT 3.3")
    print(instrument.query("SOUR:VOLT?"))
    instrument.write("OUTP ON")
    print(instrument.query("MEAS:VOLT?"))
